# Get GAV estimation for collected radical data

In [36]:
import sys
sys.path.insert(0,"/home/gridsan/hwpang/Software/RMG-Py/")

import random
import os
import time
import math
from copy import deepcopy
import pandas as pd
from joblib import Parallel, delayed
from tqdm import tqdm

from utils import load_thermo_lib_by_path, generate_thermo
from parameters import Ts

from rmgpy.data.thermo import ThermoDatabase, ThermoLibrary, ThermoData, remove_thermo_data, add_thermo_data, NASA
from rmgpy.molecule import Molecule
from rmgpy.species import Species
from rmgpy import settings
from rmgpy import constants

# Load radical data

In [2]:
path = "../data/radical.csv"
radical_data_df = pd.read_csv(path)
radical_data_df

,smiles,H298 (kcal/mol),S298 (cal/mol/K),Sint298 (cal/mol/K),source,level_of_theory,Cp300 (cal/mol/K),Cp400 (cal/mol/K),Cp500 (cal/mol/K),Cp600 (cal/mol/K),Cp800 (cal/mol/K),Cp1000 (cal/mol/K),Cp1500 (cal/mol/K),num_resonance
0,[O]C(=O)OC(O)(O)O,-223.514126,93.816804,95.999974,dong_pio_liang.py,CBS-QB3,33.926878,38.387673,41.508496,44.062390,47.779730,50.089328,52.510168,1
1,[O]C(O)(O)OC(=O)O,-223.423390,92.781436,92.781436,dong_pio_liang.py,CBS-QB3,32.531179,37.569659,41.498443,44.710460,49.355989,52.184758,54.821371,1
2,O=C(O)O[C](O)O,-187.322869,88.139642,89.517068,dong_pio_liang.py,CBS-QB3,28.973242,34.553073,38.746721,41.655671,45.237515,47.423507,49.461069,1
3,CC(=O)OC(OO)C(=O)C(O)O[O],-185.660577,133.205064,132.633380,dong_pio_liang.py,CBS-QB3,56.569703,66.028226,73.603744,79.772844,88.705511,94.174908,99.503777,1
4,O=[C]OC(O)(O)O,-180.497563,87.848805,90.031975,dong_pio_liang.py,CBS-QB3,32.378191,36.844343,39.334179,41.301130,44.143726,45.893560,47.744589,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2232,C=C=C1[CH]C1=C,148.970420,78.606070,81.360924,dong_pio_liang.py,CBS-QB3,25.185446,30.771706,34.989720,38.456637,43.926501,47.843872,53.411105,3
2233,[CH]=C1C=C1,152.166735,66.134536,67.511963,dong_pio_liang.py,CBS-QB3,16.390064,19.702535,22.399354,24.484680,27.637313,29.841300,32.891319,1
2234,C1=C[CH]C=1,156.106582,64.008341,65.385768,dong_pio_liang.py,CBS-QB3,14.653111,18.488757,21.562144,23.855179,27.322032,29.749165,33.017640,2
2235,C=C=C1C=[C]C1,157.733085,76.210198,77.587625,dong_pio_liang.py,CBS-QB3,22.675255,28.333884,32.976246,36.811783,42.847715,47.149236,53.155423,1


# Estimate GAV for radicals

In [3]:
# load thermo database

thermo_database = ThermoDatabase()
thermo_database.load_groups(os.path.join(settings["database.directory"], "thermo", "groups"))

In [4]:
generate_thermo(thermo_database, "[O]C(=O)OC(O)(O)O", resonance=True) # we want the best thermo estimation for the radical

/home/gridsan/hwpang/Software/RMG-Py/rmgpy/rmg/reactors.py:52: RuntimeWarning: Unable to import Julia dependencies, original error: No module named 'julia'
  warnings.warn("Unable to import Julia dependencies, original error: " + str(e), RuntimeWarning)


(-214.5628470363289,
 98.46305594224867,
 array([29.53076482, 33.69692161, 37.1998088 , 39.86061185, 43.9456979 ,
        46.70558317, 51.10089866]),
 102.02365200764818,
 'Thermo group additivity estimation: group(O2s-Cs(Cds-O2d)) + group(O2s-CsH) + group(O2s-CsH) + group(O2s-CsH) + group(O2s-(Cds-O2d)H) + group(Cs-OsOsOsOs) + group(Cds-OdOsOs) + radical(OC=OOJ)')

In [9]:
radical_gav_df = radical_data_df[["smiles"]]
thermos = radical_gav_df["smiles"].apply(lambda smi: generate_thermo(thermo_database, smi, resonance=True))

In [32]:
columns = ["H298 (kcal/mol)", "S298 (cal/mol/K)", "Cp (cal/mol/K)", "Sint298 (cal/mol/K)", "comment"]
radical_gav_df[columns] = thermos.to_list()

columns = []
for i, T in enumerate(Ts):
    columns.append(f"Cp{T} (cal/mol/K)")
radical_gav_df[columns] = radical_gav_df["Cp (cal/mol/K)"].to_list()

radical_gav_df = radical_gav_df.drop("Cp (cal/mol/K)", axis=1)

columns = radical_gav_df.columns.tolist()
columns.remove("comment")
columns.append("comment")

radical_gav_df = radical_gav_df[columns]

radical_gav_df

,smiles,H298 (kcal/mol),S298 (cal/mol/K),Sint298 (cal/mol/K),Cp300 (cal/mol/K),Cp400 (cal/mol/K),Cp500 (cal/mol/K),Cp600 (cal/mol/K),Cp800 (cal/mol/K),Cp1000 (cal/mol/K),Cp1500 (cal/mol/K),comment
0,[O]C(=O)OC(O)(O)O,-214.562847,98.463056,102.023652,29.530765,33.696922,37.199809,39.860612,43.945698,46.705583,51.100899,Thermo group additivity estimation: group(O2s-...
1,[O]C(O)(O)OC(=O)O,-218.506442,100.780822,100.780822,28.957151,32.645296,36.363289,39.621606,44.949522,48.354723,52.797839,Thermo group additivity estimation: group(O2s-...
2,O=C(O)O[C](O)O,-182.366843,95.359992,96.737419,26.444474,30.605583,34.122333,36.909560,40.899044,43.401300,45.798700,Thermo group additivity estimation: group(O2s-...
3,CC(=O)OC(OO)C(=O)C(O)O[O],-198.122312,145.818548,145.246864,50.656616,59.598776,67.892505,73.980363,82.723365,90.133078,97.599981,Thermo group additivity estimation: group(O2s-...
4,O=[C]OC(O)(O)O,-169.366862,91.656926,93.840096,27.460975,30.914895,33.801147,36.038910,39.770268,42.499082,46.915908,Thermo group additivity estimation: group(O2s-...
...,...,...,...,...,...,...,...,...,...,...,...,...
2232,C=C=C1[CH]C1=C,136.664974,51.873385,51.873385,21.878919,28.083266,33.062461,37.060145,42.684263,46.372044,52.585950,Thermo group additivity estimation: group(Cs-(...
2233,[CH]=C1C=C1,159.833,64.598273,65.9757,16.187000,19.787000,22.505000,24.576000,27.584000,29.724000,33.028000,Thermo group additivity estimation: group(Cds-...
2234,C1=C[CH]C=1,148.2005,63.142873,64.5203,12.454000,16.060000,19.314000,22.291000,26.629000,29.639000,34.394000,Thermo group additivity estimation: group(Cs-(...
2235,C=C=C1C=[C]C1,151.763874,52.204958,53.582385,22.076919,28.279266,33.060461,36.665145,41.822263,45.330044,50.563950,Thermo group additivity estimation: group(Cs-(...


In [33]:
radical_gav_df.to_csv("../data/radical_gav.csv", index=False)

# Plot: compare radical data and GAV estimation

In [37]:
radical_data_df = pd.read_csv("../data/radical.csv")
radical_gav_df = pd.read_csv("../data/radical_gav.csv")

In [40]:
fig, axs = plt.subplots(nrows=3, ncols=3, sharey=True, figsize=(9, 7))
ax = axs.flat[0]
ax.hist(radical_gav_df["H298 (kcal/mol)"] - radical_data_df["H298 (kcal/mol)"], bins=50)
ax.set_xlabel("$\Delta H^\circ_\mathrm{f,298}$ (kcal/mol)")

ax = axs.flat[1]
ax.hist(radical_gav_df["S298 (cal/mol/K)"] - radical_data_df["S298 (cal/mol/K)"], bins=50)
ax.set_xlabel("$S^\circ_\mathrm{298}$ (cal/mol/K)")

for ax, T in zip(axs.flat[2:], Ts):
    ax.hist(radical_gav_df[f"Cp{T} (cal/mol/K)"] - radical_data_df[f"Cp{T} (cal/mol/K)"], bins=50)
    ax.set_xlabel("$C_\mathrm{p,{" + str(T) + "}}$ (cal/mol/K)")

fig.supylabel(f"Count (N={len(radical_data_df.index)})")
fig.supxlabel(f"Error (QM - GAV)")

fig.tight_layout()
fig.show()

/home/gridsan/hwpang/mambaforge/envs/rmg_rdmc_env_20230623_v2/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
